In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# this is needed in azure
sc = SparkContext( 'spark://headnodehost:7077', 'pyspark')

In [14]:
lines = sc.textFile("wasb://spark-meetup@analyticsdatasets.blob.core.windows.net/example/data/gutenberg/*")
#data_rdd = sc.textFile("s3n://aws-publicdatasets/common-crawl/crawl-data/CC-MAIN-2015-32/segments/1438042981460.12/wet/*")
lines.setName("lines") # This is needed to see the name in the DAG

lines MapPartitionsRDD[15] at textFile at NativeMethodAccessorImpl.java:-2

In [ ]:
lines.count()

In [ ]:
words = lines.flatMap(lambda x: x.split()) \
             .filter(lambda x: len(x)>0) \
             .map(lambda x: x.lower())
words.setName("words")
words.count()

In [ ]:
word_counts = ( words.map(lambda x: (x,1))
                      .reduceByKey(lambda x,y: x+y)
                      .takeOrdered(200, key = lambda (x,y): -y) )
word_counts